# Word2Vec

In [2]:
import numpy as np
import pandas as pd

In [3]:
temp_df = pd.read_csv(r"C:\Users\prajwal\Downloads\imdb_dataset\IMDB Dataset.csv")

In [4]:
df = temp_df.iloc[:10000]

In [5]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [6]:
df.drop_duplicates(inplace=True)

C:\Users\prajwal\AppData\Local\Temp\ipykernel_8064\3006716147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


In [7]:
import re
def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>'), '', raw_text)
    return cleaned_text

In [8]:
df['review'] = df['review'].apply(remove_tags)

C:\Users\prajwal\AppData\Local\Temp\ipykernel_8064\2336150696.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(remove_tags)


In [9]:
df['review'] = df['review'].apply(lambda x:x.lower())

C:\Users\prajwal\AppData\Local\Temp\ipykernel_8064\740760900.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x:x.lower())


In [10]:
from nltk.corpus import stopwords

sw_list = stopwords.words('english')

df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

C:\Users\prajwal\AppData\Local\Temp\ipykernel_8064\2826946130.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))


In [11]:
df['review']

0       one reviewers mentioned watching 1 oz episode ...
1       wonderful little production. filming technique...
2       thought wonderful way spend time hot summer we...
3       basically there's family little boy (jake) thi...
4       petter mattei's "love time money" visually stu...
                              ...                        
9995    fun, entertaining movie wwii german spy (julie...
9996    give break. anyone say "good hockey movie"? kn...
9997    movie bad movie. watching endless series bad h...
9998    movie probably made entertain middle school, e...
9999    smashing film film-making. shows intense stran...
Name: review, Length: 9983, dtype: object

In [16]:
import gensim

In [17]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [23]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [24]:
story = []
for doc in df['review']:
    raw_sent = sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))

In [25]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2
)

In [26]:
model.build_vocab(story)

In [27]:
model.train(story, total_examples=model.corpus_count, epochs=model.epochs)

(5875953, 6212140)

In [28]:
len(model.wv.index_to_key)

31845

In [29]:
def document_vector(doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc], axis=0)

In [30]:
document_vector(df['review'].values[0])

array([-1.7436297e-01,  4.6271846e-01,  1.2986171e-01,  2.3855846e-01,
       -1.1641364e-01, -6.0518420e-01,  2.0777844e-01,  9.9462223e-01,
       -3.1645158e-01, -2.7232265e-01, -3.1547272e-01, -5.3179103e-01,
        1.3771623e-01,  1.0970077e-01,  1.4488515e-01, -1.8100937e-01,
        3.9682180e-02, -3.5624573e-01, -2.2796858e-02, -6.5261126e-01,
       -5.8758883e-03,  2.1823914e-01,  1.4079382e-01, -2.9852349e-01,
       -2.4515404e-01, -5.0454263e-02, -3.3012477e-01, -7.7139569e-04,
       -3.0297366e-01,  1.5378711e-02,  3.1131357e-01, -4.3682922e-02,
        2.4163792e-01, -2.5166246e-01, -1.8549258e-01,  4.0659916e-01,
        1.9586556e-01, -3.8989073e-01, -1.9951335e-01, -7.0896769e-01,
        1.3877997e-01, -2.3428573e-01,  6.9825307e-02, -8.6977936e-02,
        5.4396117e-01, -8.0954991e-02, -2.5787318e-01,  1.2733199e-03,
        8.9181580e-02,  3.2536411e-01,  5.5638235e-02, -3.2032096e-01,
       -4.2747098e-01, -1.3422461e-01, -6.7854866e-02,  1.3487555e-01,
      

In [31]:
from tqdm import tqdm

In [32]:
X = []
for doc in tqdm(df['review'].values):
    X.append(document_vector(doc))

100%|██████████████████████████████████████████████████████████████████████████████| 9983/9983 [15:42<00:00, 10.59it/s]


In [33]:
X = np.array(X)

In [34]:
X[0]

array([-1.7436297e-01,  4.6271846e-01,  1.2986171e-01,  2.3855846e-01,
       -1.1641364e-01, -6.0518420e-01,  2.0777844e-01,  9.9462223e-01,
       -3.1645158e-01, -2.7232265e-01, -3.1547272e-01, -5.3179103e-01,
        1.3771623e-01,  1.0970077e-01,  1.4488515e-01, -1.8100937e-01,
        3.9682180e-02, -3.5624573e-01, -2.2796858e-02, -6.5261126e-01,
       -5.8758883e-03,  2.1823914e-01,  1.4079382e-01, -2.9852349e-01,
       -2.4515404e-01, -5.0454263e-02, -3.3012477e-01, -7.7139569e-04,
       -3.0297366e-01,  1.5378711e-02,  3.1131357e-01, -4.3682922e-02,
        2.4163792e-01, -2.5166246e-01, -1.8549258e-01,  4.0659916e-01,
        1.9586556e-01, -3.8989073e-01, -1.9951335e-01, -7.0896769e-01,
        1.3877997e-01, -2.3428573e-01,  6.9825307e-02, -8.6977936e-02,
        5.4396117e-01, -8.0954991e-02, -2.5787318e-01,  1.2733199e-03,
        8.9181580e-02,  3.2536411e-01,  5.5638235e-02, -3.2032096e-01,
       -4.2747098e-01, -1.3422461e-01, -6.7854866e-02,  1.3487555e-01,
      

In [35]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

y = encoder.fit_transform(df['sentiment'])

In [36]:
y

array([1, 1, 1, ..., 0, 0, 1])

In [37]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [38]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [39]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.771156735102654